In [1]:
import numpy as np
import argparse
from datasets.config import ModelConfig
from modules.basic_nn import BasicRS
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import FNNData
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm

In [2]:
set_seed(7)
dpath = 'data'

In [3]:
trainp = os.path.join(dpath, 'train.npy')
devp = os.path.join(dpath, 'valid.npy')

In [4]:
trainset = FNNData(np.load(trainp))
validset = FNNData(np.load(devp))

In [5]:
train_data_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
valid_data_loader = DataLoader(validset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [6]:
model = BasicRS(ModelConfig())

In [7]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    loss_list = []
    for data in enum_dataloader:
        data = data.type(torch.LongTensor)
        pred = model(data[:, 2:])
        loss = F.cross_entropy(pred, data[:, 1])
        loss_list.append(loss)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))

EP-0 train loss: 1.1370081901550293: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 4185/4185 [00:48<00:00, 86.79it/s]


epoch 0 end


EP-1 train loss: 1.0683177709579468:  35%|███████████████████████████████████▌                                                                 | 1473/4185 [00:42<00:50, 53.63it/s]